In [2]:
%pip install --upgrade --quiet tiktoken langchain-openai python-dotenv datasets langchain beautifulsoup4 html2text ragas "deeplake[enterprise]<4.0.0"


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv


load_dotenv("../.env")
assert os.getenv("OPENAI_API_KEY")
assert os.getenv("ACTIVELOOP_TOKEN")
assert os.getenv("LANGCHAIN_API_KEY")
assert os.getenv("LANGCHAIN_TRACING_V2")
assert os.getenv("LANGCHAIN_ENDPOINT")
assert os.getenv("LANGCHAIN_PROJECT")

In [2]:
# Load docs
from langchain.document_loaders import WebBaseLoader


loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()
len(data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


1

In [3]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)
len(all_splits)

133

In [4]:
# Store splits
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import DeepLake


vectorstore = DeepLake.from_documents(documents=all_splits, dataset_path="hub://yaroslava/langsmith_intro", embedding=OpenAIEmbeddings(), overwrite=True)

/Users/coder/Desktop/dev/rag_for_production_activeloop/rag-for-production-with-langchain-and-llamaindex/.venv/lib/python3.12/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (4.3.1) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(
/var/folders/nl/9vvy243s0cgd8rww2jfgd9jw0000gn/T/ipykernel_3438/4054566647.py:6: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  vectorstore = DeepLake.from_documents(documents=all_splits, dataset_path="hub://yaroslava/langsmith_intro", embedding=OpenAIEmbeddings(), overwrite=True)


Your Deep Lake dataset has been successfully created!


Creating 133 embeddings in 1 batches of size 133:: 100%|██████████| 1/1 [00:12<00:00, 12.64s/it]

Dataset(path='hub://yaroslava/langsmith_intro', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype       shape      dtype  compression
  -------    -------     -------    -------  ------- 
   text       text      (133, 1)      str     None   
 metadata     json      (133, 1)      str     None   
 embedding  embedding  (133, 1536)  float32   None   
    id        text      (133, 1)      str     None   


In [5]:
# RAG prompt
from langchain import hub


prompt = hub.pull("rlm/rag-prompt:50442af1")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [6]:
# LLM
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI


llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

/var/folders/nl/9vvy243s0cgd8rww2jfgd9jw0000gn/T/ipykernel_3438/3053291969.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


In [7]:
# RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [8]:
question = "What are the approaches to Task Decomposition?"
result = qa_chain({"query": question})
result["result"]

/var/folders/nl/9vvy243s0cgd8rww2jfgd9jw0000gn/T/ipykernel_3438/1574984886.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


'The approaches to Task Decomposition include using LLM with simple prompting, task-specific instructions, and human inputs. Task decomposition involves breaking down large tasks into smaller subgoals for efficient handling of complex tasks and reflecting on past actions to improve future results. Challenges in long-term planning and task decomposition include adjusting plans in the face of unexpected errors, which LLMs struggle with compared to humans.'

# Environment

In [9]:
%pip list

Package                  Version
------------------------ -----------
aioboto3                 15.1.0
aiobotocore              2.24.0
aiofiles                 24.1.0
aiohappyeyeballs         2.6.1
aiohttp                  3.12.15
aioitertools             0.12.0
aiosignal                1.4.0
annotated-types          0.7.0
anyio                    4.10.0
appdirs                  1.4.4
appnope                  0.1.4
asttokens                3.0.0
attrs                    25.3.0
beautifulsoup4           4.13.5
boto3                    1.39.11
botocore                 1.39.11
certifi                  2025.8.3
charset-normalizer       3.4.3
click                    8.2.1
comm                     0.2.3
dataclasses-json         0.6.7
datasets                 4.0.0
debugpy                  1.8.16
decorator                5.2.1
deepframe                0.1.1
deeplake                 3.9.52
dill                     0.4.0
diskcache                5.6.3
distro                   1.9.0
docstring_par